In [1]:
#import all libraries
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
import os
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from nltk.tokenize import WordPunctTokenizer

In [2]:
os.chdir('../')
print(os.listdir())
print(os.getcwd())

['.ipynb_checkpoints', 'Dataset Generator.ipynb', 'dataset.xlsx', 'latest_dataset.xlsx', 'Pre-Process', 'queries.xlsx']
C:\Users\User\Desktop\Chatbot\Project Working\ML, Cognitive, Human


In [3]:
df = pd.read_excel('latest_dataset.xlsx',usecols = "A,B")

In [5]:
df.dropna(inplace=True)

In [4]:
#initiliaze attributes
lemmatizer = WordNetLemmatizer()
stop_words = set (stopwords.words('english'))
tok = WordPunctTokenizer()
queries = []

for query in df['query']:
    #remove all links
    data = re.sub(r"@\S*|#|http\S+", "", query)
    #then apply lemmatization
    data1 = data.split()
    newdata = ""
    for data in data1:
        datas = lemmatizer.lemmatize(data)
        newdata = newdata + " " + datas

    #then finally check for stopwords
    words_token = word_tokenize(newdata)
    #filtered_sentence = [w for w in words_token if not w in stop_words]
    filtered_sentence = ""
    for w in words_token:
        if w not in stop_words:
            filtered_sentence = filtered_sentence + " " + w

    queries.append(filtered_sentence)
    df.dropna(inplace=True)


#definig regular expressions
pat1 = r'@[A-Za-z0-9]+'
pat2 = r'https?://[A-Za-z0-9./]+'
combined_pat = r'|'.join((pat1, pat2))

#creating a function which does all the work
def query_cleaner(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    stripped = re.sub(combined_pat, '', souped)
    try:
        clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        clean = stripped
    #only letters remove other characters
    letters_only = re.sub("[^a-zA-Z]", " ", clean)
    lower_case = letters_only.lower()
    words = tok.tokenize(lower_case)

    return (" ".join(words)).strip()

#running the function and returns the result in dataframe
queries_new = []
for query in queries:
    queries_new.append(query_cleaner(query))

#append it to the dataframe
df['query'] = queries_new
df.dropna(inplace=True, how='any')


In [5]:
os.chdir('pre-process')

In [6]:
os.getcwd()

'C:\\Users\\User\\Desktop\\Chatbot\\Project Working\\ML, Cognitive, Human\\pre-process'

In [7]:
#output dataframe to excel format
df.to_excel('cleaned_queries.xlsx',index=None)